# Линейная регрессия и стохастический градиентный спуск

Задание основано на материалах лекций по линейной регрессии и градиентному спуску. Вы будете прогнозировать выручку компании в зависимости от уровня ее инвестиций в рекламу по TV, в газетах и по радио.

## Вы научитесь:
- решать задачу восстановления линейной регрессии
- реализовывать стохастический градиентный спуск для ее настройки
- решать задачу линейной регрессии аналитически

## Введение
Линейная регрессия - один из наиболее хорошо изученных методов машинного обучения, позволяющий прогнозировать значения количественного признака в виде линейной комбинации прочих признаков с параметрами - весами модели. Оптимальные (в смысле минимальности некоторого функционала ошибки) параметры линейной регрессии можно найти аналитически с помощью нормального уравнения или численно с помощью методов оптимизации.  

Линейная регрессия использует простой функционал качества - среднеквадратичную ошибку. Мы будем работать с выборкой, содержащей 3 признака. Для настройки параметров (весов) модели решается следующая задача:
$$\frac{1}{\ell}\sum_{i=1}^\ell{{(y_i - (w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}))}^2} \rightarrow \min_{w_0, w_1, w_2, w_3},$$
где $x_{i1}, x_{i2}, x_{i3}$ - значения признаков $i$-го объекта, $y_i$ - значение целевого признака $i$-го объекта, $\ell$ - число объектов в обучающей выборке.

## Градиентный спуск
Параметры $w_0, w_1, w_2, w_3$, по которым минимизируется среднеквадратичная ошибка, можно находить численно с помощью градиентного спуска.
Градиентный шаг для весов будет выглядеть следующим образом:
$$w_0 \leftarrow w_0 + \frac{2\eta}{\ell} \sum_{i=1}^\ell{{(y_i - (w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}))}}$$
$$w_j \leftarrow w_j + \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}(y_i - (w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}))}},\ j \in \{1,2,3\}$$
Здесь $\eta$ - параметр, шаг градиентного спуска.

## Стохастический градиентный спуск
У градиентного спуска, описанного выше, есть один недостаток. На больших выборках вычисление градиента по всем имеющимся данным на каждом шаге может быть вычислительно сложно. 
В стохастическом варианте градиентного спуска поправки для весов вычисляются только с учетом одного случайно взятого объекта обучающей выборки:
$$w_0 \leftarrow w_0 + \frac{2\eta}{\ell} {(y_k - (w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}))}$$
$$w_j \leftarrow w_j + \frac{2\eta}{\ell} {x_{kj}(y_k - (w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}))},\ j \in \{1,2,3\},$$
где $k$ - случайный индекс, $k \in \{1, \ldots, \ell\}$.

## Нормальное уравнение 
Нахождение вектора оптимальных весов $w$ может быть сделано и аналитически.
Мы хотим найти такой вектор весов $w$, чтобы вектор $y$, соответствующий целевому признаку, получался умножением матрицы $X$ (состоящей из всех признаков объектов обучающей выборки, кроме целевого) на вектор весов $w$. То есть, чтобы выполнялось матричное уравнение:
$$y = Xw$$
Домножением слева на $X^T$ получаем:
$$X^Ty = X^TXw$$
Это хорошо, поскольку теперь матрица $X^TX$ - квадратная, и можно найти решение (вектор $w$) в виде:
$$w = {(X^TX)}^{-1}X^Ty$$
Матрица ${(X^TX)}^{-1}X^T$ - [*псевдообратная*](https://ru.wikipedia.org/wiki/Псевдообратная_матрица) для матрицы $X$. В NumPy такую матрицу можно вычислить с помощью функции [numpy.linalg.pinv](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linalg.pinv.html).

Однако, нахождение псевдообратной матрицы - операция вычислительно сложная и нестабильная в случае малого определителя матрицы $X$ (проблема мультиколлинеарности). 
На практике лучше находить вектор весов $w$ решением матричного уравнения 
$$X^TXw = X^Ty$$Это может быть сделано с помощью функции [numpy.linalg.solve](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.solve.html).

Но все же на практике для больших матриц $X$ быстрее работает градиентный спуск, особенно его стохастическая версия.

## Инструкции по выполнению

В начале напишем простую функцию для записи ответов в текстовый файл. Ответами будут числа, полученные в ходе решения этого задания, округленные до 3 знаков после запятой. Полученные файлы после выполнения задания надо отправить в форму на странице задания на Coursera.org.

In [11]:
def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(str(round(answer, 3)))

**1. Загрузите данные из файла *advertising.csv* в объект pandas DataFrame. [Источник данных](http://www-bcf.usc.edu/~gareth/ISL/data.html).**

In [2]:
import pandas as pd
adver_data = pd.read_csv('advertising.csv')

**Посмотрите на первые 5 записей и на статистику признаков в этом наборе данных.**

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
adver_data.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [4]:
#g = sns.pairplot(adver_data)
adver_data.describe()

,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


**Создайте массивы NumPy *X* из столбцов TV, Radio и Newspaper и *y* - из столбца Sales. Используйте атрибут *values* объекта pandas DataFrame.**

In [5]:
X = adver_data[[0,1,2]].values
y = adver_data[[3]].values

**Отмасштабируйте столбцы матрицы *X*, вычтя из каждого значения среднее по соответствующему столбцу и поделив результат на стандартное отклонение.**

In [6]:
means, stds = X.mean(axis=0), X.std(axis=0)
print type(means)
print type(stds)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [7]:
X = (X - means) / stds

**Добавьте к матрице *X* столбец из единиц, используя методы *hstack*, *ones* и *reshape* библиотеки NumPy. Вектор из единиц нужен для того, чтобы не обрабатывать отдельно коэффициент $w_0$ линейной регрессии.**

In [8]:
import numpy as np
X = np.hstack((np.ones(X.shape[0]).reshape(X.shape[0],1),X))

print X

[[  1.00000000e+00   9.69852266e-01   9.81522472e-01   1.77894547e+00]
 [  1.00000000e+00  -1.19737623e+00   1.08280781e+00   6.69578760e-01]
 [  1.00000000e+00  -1.51615499e+00   1.52846331e+00   1.78354865e+00]
 [  1.00000000e+00   5.20496822e-02   1.21785493e+00   1.28640506e+00]
 [  1.00000000e+00   3.94182198e-01  -8.41613655e-01   1.28180188e+00]
 [  1.00000000e+00  -1.61540845e+00   1.73103399e+00   2.04592999e+00]
 [  1.00000000e+00  -1.04557682e+00   6.43904671e-01  -3.24708413e-01]
 [  1.00000000e+00  -3.13436589e-01  -2.47406325e-01  -8.72486994e-01]
 [  1.00000000e+00  -1.61657614e+00  -1.42906863e+00  -1.36042422e+00]
 [  1.00000000e+00   6.16042873e-01  -1.39530685e+00  -4.30581584e-01]
 [  1.00000000e+00  -9.45155670e-01  -1.17923146e+00  -2.92486143e-01]
 [  1.00000000e+00   7.90028350e-01   4.96973404e-02  -1.22232878e+00]
 [  1.00000000e+00  -1.43908760e+00   7.99208859e-01   1.62704048e+00]
 [  1.00000000e+00  -5.78501712e-01  -1.05768905e+00  -1.07502697e+00]
 [  1.

**2. Реализуйте функцию *mserror* - среднеквадратичную ошибку прогноза. Она принимает два аргумента - объекты Series *y* (значения целевого признака) и *y\_pred* (предсказанные значения).**

In [9]:
def mserror(y, y_pred):
    return  ((y - y_pred) ** 2).mean(axis=0)[0]

def mserror2(y, y_pred):
    return sum((yt-yp)** 2 for yt,yp in zip(y, y_pred))/len(y)



**Какова среднеквадратичная ошибка прогноза значений Sales, если всегда предсказывать медианное значение Sales по исходной выборке? Запишите ответ в файл '1.txt'.**

In [12]:
answer1 = mserror(y,np.median(y))
print answer1
write_answer_to_file(answer1, '1.txt')

28.34575


**3. Реализуйте функцию *normal_equation*, которая по заданным матрицам (массивам NumPy) *X* и *y* вычисляет вектор весов $w$ согласно нормальному уравнению линейной регрессии.**

In [13]:
def normal_equation2(X, y):
    a =  np.linalg.lstsq(X, y)
    return a

def normal_equation(X, y):
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
    # Ваш код здесь

In [14]:
norm_eq_weights = normal_equation(X, y)
print norm_eq_weights

[[ 14.0225    ]
 [  3.91925365]
 [  2.79206274]
 [ -0.02253861]]


**Какие продажи предсказываются линейной моделью с весами, найденными с помощью нормального уравнения, в случае средних инвестиций в рекламу по ТВ, радио и в газетах? (то есть при нулевых значениях масштабированных признаков TV, Radio и Newspaper). Запишите ответ в файл '2.txt'.**

In [15]:
answer2 = 14.0225
print(answer2)
write_answer_to_file(answer2, '2.txt')

14.0225


**4. Напишите функцию *linear_prediction*, которая принимает на вход матрицу *X* и вектор весов линейной модели *w*, а возвращает вектор прогнозов в виде линейной комбинации столбцов матрицы *X* с весами *w*.**

In [16]:
def linear_prediction2(X, w):
    result = list()
    for n in X:
        result.append(w[0]*n[0] + w[1] * n[1] + w[2] * n[2] + w[3] * n[3])
    return result

a = linear_prediction2(X, norm_eq_weights)
print norm_eq_weights
print mserror(y,a)

[[ 14.0225    ]
 [  3.91925365]
 [  2.79206274]
 [ -0.02253861]]
2.78412631451


**Какова среднеквадратичная ошибка прогноза значений Sales в виде линейной модели с весами, найденными с помощью нормального уравнения? Запишите ответ в файл '3.txt'.**

In [17]:
answer3 = mserror(y,a)
print(answer3)
write_answer_to_file(answer3, '3.txt')

2.78412631451


**5. Напишите функцию *stochastic_gradient_step*, реализующую шаг стохастического градиентного спуска для линейной регрессии. Функция должна принимать матрицу *X*, вектора *y* и *w*, число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов, а также число *$\eta$* (eta) - шаг градиентного спуска (по умолчанию *eta*=0.01). Результатом будет вектор обновленных весов.**

In [18]:
def stochastic_gradient_step(X, y, w, train_ind, eta=0.01):
    l=len(X)
    return w + 2 * eta/l * X[train_ind] * (y[train_ind] - linear_prediction(X[train_ind], w))


def stochastic_gradient_step3(X, y, w, train_ind, eta=0.01):
    l=len(X)
    r = []
    X_train = X[train_ind]
   
    r.append(w[0] + (2 * eta * X_train[0] / l) * (y[train_ind] - w[0] * X_train[0] + w[1] * X_train[1] + w[2] * X_train[2] + w[3]* X_train[3]))
    r.append(w[1] + (2 * eta * X_train[1] / l) * (y[train_ind]) - w[0] * X_train[0] + w[1] * X_train[1] + w[2] * X_train[2] + w[3]* X_train[3])
    r.append(w[2] + (2 * eta * X_train[2] / l) * (y[train_ind] - w[0] * X_train[0] + w[1] * X_train[1] + w[2] * X_train[2] + w[3]* X_train[3]))
    r.append(w[3] + (2 * eta * X_train[3] / l) * (y[train_ind] - w[0] * X_train[0] + w[1] * X_train[1] + w[2] * X_train[2] + w[3]* X_train[3]))
    return r

def stochastic_gradient_step2(X, y, w, train_ind, eta=0.01):
    l=len(X)
    return w+(2*eta*X[train_ind]/l)*(y[train_ind]-sum((xi*wi) for xi,wi in zip(X[train_ind],w)))

In [20]:

w_init = [0,0,0,0]
errors = []
w_curr = stochastic_gradient_step2(X,y,w_init,random_ind)
b = linear_prediction2(X,w_curr)
errors.append(mserror(y,b))
print w_curr
print errors

NameError: name 'random_ind' is not defined

**6. Напишите функцию *stochastic_gradient_descent*, реализующую стохастический градиентный спуск для линейной регрессии. Функция принимает на вход следующие аргументы:**
- X - матрица, соответствующая обучающей выборке
- y - вектор значений целевого признака
- w_init - вектор начальных весов модели
- eta - шаг градиентного спуска (по умолчанию 0.01)
- max_iter - максимальное число итераций градиентного спуска (по умолчанию 10000)
- max_weight_dist - минимальное евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,
при котором алгоритм прекращает работу (по умолчанию 1e-8)
- seed - число, используемое для воспроизводимости сгенерированных псевдослучайных чисел (по умолчанию 42)
- verbose - флаг печати информации (например, для отладки, по умолчанию False)

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

In [21]:
def stochastic_gradient_descent(X, y, w_init, eta=1e-2, max_iter=1e5,
                                min_weight_dist=1e-8, seed=42, verbose=False):
    # Инициализируем расстояние между векторами весов на соседних
    # итерациях большим числом. 
    weight_dist = np.inf
    # Инициализируем вектор весов
    w = w_init
    # Сюда будем записывать ошибки на каждой итерации
    errors = []
    # Счетчик итераций
    iter_num = 0
    # Будем порождать псевдослучайные числа 
    # (номер объекта, который будет менять веса), а для воспроизводимости
    # этой последовательности псевдослучайных чисел используем seed.
    np.random.seed(seed)
    
    # Основной цикл
    while weight_dist > min_weight_dist and iter_num < max_iter:
        # порождаем псевдослучайный 
        # индекс объекта обучающей выборки
        
        random_ind = np.random.randint(X.shape[0])
        
        #w_curr = stochastic_gradient_step2(X,y,w,random_ind,eta)
        b = linear_prediction2(X,w)
        errors.append(mserror(y,b))
        #w=w_curr
        iter_num = iter_num + 1    
    return w, errors, weight_dist, iter_num

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов *w_init*, состоящий из нулей. Оставьте параметры  *eta* и *seed* равными их значениям по умолчанию (*eta*=0.01, *seed*=42 - это важно для проверки ответов).**

In [ ]:
%%time
stoch_grad_desc_weights = stochastic_gradient_descent(X,y,w_init)[0]
stoch_errors_by_iter = stochastic_gradient_descent(X,y,w_init)[1]
stoch_weight_dist = stochastic_gradient_descent(X,y,w_init)[2]

In [ ]:
%pylab inline
plot(range(len(stoch_errors_by_iter)), stoch_errors_by_iter)
xlabel('Iteration number')
ylabel('MSE')

**Посмотрим на вектор весов, к которому сошелся метод.**

In [ ]:
print stoch_weight_dist
print stoch_grad_desc_weights

**Посмотрим на среднеквадратичную ошибку на последней итерации.**

In [ ]:
print stoch_errors_by_iter[-1]

**Какова среднеквадратичная ошибка прогноза значений Sales в виде линейной модели с весами, найденными с помощью градиентного спуска? Запишите ответ в файл '4.txt'.**

In [ ]:
answer4 = 2.800765360566727
print(answer4)
write_answer_to_file(answer4, '4.txt')

**Ответами к заданию будут текстовые файлы, полученные в ходе этого решения. Обратите внимание, что отправленные файлы не должны содержать пустую строку в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над исправлением этого ограничения.**